In [3]:
import httpx
import json
from joblib import Memory
from bs4 import BeautifulSoup
memory = Memory("./cache", verbose=0)

@memory.cache
def fetch_data(url : str) -> str:
    
    cookies = {
        'req-device-token': '7de4109a-0e61-4488-b544-b451d2321403',
        'cf_clearance': 'wLDXjJ.KdsclQ7VCV72DxifaOwmOoKWRiKFauzNLulo-1766670311-1.2.1.1-estcFeioX18Hn3NLEbhu3_cN3NEMyW6HZ4cIbs9DdErQX2HnejhNNcuah8dHPiaMig.n0bHutzNz12u9kxluSJDyqadLqIQtPAW.x5iL9mu8E3YT3CZxLG_mQRqhJ25Yn.K6PT_EfgpT4y65biIg34WnurMVcLhe0lNE0hxPLUWr8JnUqbx5UyOclmQNBlBluOKK0u.OgNX8oAvRgOFdf9KHK2SvZAafGKaz4jbbf34',
        '__cf_bm': 'L2zIqLFZrqYT3yludtz_cx3IAmkqGIZNFYALmaicyEk-1766670979-1.0.1.1-ebldrairypvv6uBJWyogayDUj2bOcz.TJMAJ0CdDRrOx8bW.lVU95FldiIQPIU_Zsfw9E3F_Vzuhj2dwUduWNA90jONogjXrIX80drruYzE',
    }

    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-US,en;q=0.9,ar;q=0.8',
        'cache-control': 'no-cache',
        'pragma': 'no-cache',
        'priority': 'u=0, i',
        'referer': 'https://duckduckgo.com/',
        'sec-ch-ua': '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Linux"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36',
        # 'cookie': 'req-device-token=7de4109a-0e61-4488-b544-b451d2321403; cf_clearance=wLDXjJ.KdsclQ7VCV72DxifaOwmOoKWRiKFauzNLulo-1766670311-1.2.1.1-estcFeioX18Hn3NLEbhu3_cN3NEMyW6HZ4cIbs9DdErQX2HnejhNNcuah8dHPiaMig.n0bHutzNz12u9kxluSJDyqadLqIQtPAW.x5iL9mu8E3YT3CZxLG_mQRqhJ25Yn.K6PT_EfgpT4y65biIg34WnurMVcLhe0lNE0hxPLUWr8JnUqbx5UyOclmQNBlBluOKK0u.OgNX8oAvRgOFdf9KHK2SvZAafGKaz4jbbf34; __cf_bm=L2zIqLFZrqYT3yludtz_cx3IAmkqGIZNFYALmaicyEk-1766670979-1.0.1.1-ebldrairypvv6uBJWyogayDUj2bOcz.TJMAJ0CdDRrOx8bW.lVU95FldiIQPIU_Zsfw9E3F_Vzuhj2dwUduWNA90jONogjXrIX80drruYzE',
    }

    response = httpx.get(url,cookies=cookies, headers=headers,timeout=30,follow_redirects=True)
    textof=response.text
    
    assert not "you have been blocked" in textof.lower(), "Blocked by the website"
    return textof

def get_all_pages(url: str) -> list[str]:
    all_pages = []
    page_number = 1

    while True:
        paged_url = f"{url}/{page_number}"
        print(f"Fetching page {page_number}: {paged_url}")
        page_content = fetch_data(paged_url)
        if "No results found" in page_content:
            break
        all_pages.append(page_content)
        page_number += 1

    return all_pages

def parse_page(page:str)-> list[dict]:
    output=[]
    soup = BeautifulSoup(page, "html.parser")
    listings = soup.select("#__next > main > div > div._root__Szbd6 > div > div:nth-child(2) > div._container__Lu67A > div._list__Ka30R > div")    
    for listing in listings:
        dict_item={}
        dict_item["title"]=listing.select_one("a > div > div._content__W4gas > div._titleRow__1AWv1 > h4")
        dict_item["price"]=listing.select_one("a > div > div._content__W4gas > div._price__X51mi > span")
        dict_item["description"]=listing.select_one("a > div > div._content__W4gas > div._description__zVaD6")
        dict_item["city"]=listing.select_one("a > div > div._content__W4gas > div._footer__CnldH > p > span:nth-child(1)")
        dict_item["neighborhood"]=listing.select_one("a > div > div._content__W4gas > div._footer__CnldH > p > span:nth-child(2)")
        dict_item["area"] = listing.select_one("a > div > div._content__W4gas > div._specs__nbsgm > div:nth-child(1)")
        dict_item["bedrooms"] = listing.select_one("a > div > div._content__W4gas > div._specs__nbsgm > div:nth-child(2)")
        dict_item["bathrooms"] = listing.select_one("a > div > div._content__W4gas > div._specs__nbsgm > div:nth-child(3)")
        output.append(dict_item)
    return output
    

    
    
    
request = fetch_data("https://sa.aqar.fm/%D8%B9%D9%82%D8%A7%D8%B1%D8%A7%D8%AA/")
soup = BeautifulSoup(request, "html.parser")
out=parse_page(request)
print(out)

[{'title': <h4>دور للبيع في شارع شهيد الدين ثم الوطن فهد إبراهيم علي أب, حي النرجس, مدينة الرياض, منطقة الرياض</h4>, 'price': <span>1,500,000 <span class="icon-NewSaudiCurrency"></span></span>, 'description': <div class="_description__zVaD6"><p>مشروع ادوار سِـــدرا مشروع يتميز بموقعه الاستراتجي بحي النرجس جنوب طريق انس بن مالك وتصاميمه الفريدة.

دور ثاني بمساحة 169م

محتويات الدور:
-غرفة نوم ماستر 
-غرفة نوم 1
-غرفة نوم 2
-غرفة خادمة بدورة مياه
-صالة
-مطبخ
-سطح امامي 
-بلكونة عند الغرفة الماستر
-3 دورات مياه</p></div>, 'city': <span>الرياض</span>, 'neighborhood': <span>-حي النرجس</span>, 'area': <div class="_spec__SIJiK"><span class="icon_icon___L1OO"><img alt="المساحة" data-nimg="1" decoding="async" fetchpriority="high" height="20" src="https://assets.aqar.fm/icons/v1/area.svg" style="color:transparent;width:20px;height:20px" width="20"/></span>116 م²</div>, 'bedrooms': <div class="_spec__SIJiK"><span class="icon_icon___L1OO"><img alt="عدد الغرف" data-nimg="1" decoding="async" fetchpr